<a href="https://colab.research.google.com/github/asmabahrini453/HexSoftwares_AI_Chatbot_Development/blob/main/FIREBOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
#libraries
import numpy as np
import nltk #for handeling  NLP tasks
import string #for string manipulation
import random

from sklearn.feature_extraction.text import TfidfVectorizer #convert text to numerical vectors  based on the Term Frequency-Inverse Document Frequency (TF-IDF) method.
from sklearn.metrics.pairwise import cosine_similarity #Measures the similarity between two vectors.

In [50]:
#Read the text
txt = open('/content/sample_data/fire.txt','r',errors= 'ignore')
raw_doc = txt.read()

In [51]:
raw_doc

'Fire in Your Workspace\n\nIf you have a safe evacuation route (minimal or no fire and/or smoke) from the area:\n\n- Stay calm and breathe deeply. Remember that panic can make it harder to think clearly.\n- Evacuate the building: to the designated emergency assembly area at a steady pace; there is no need to rush.\n- Pull the fire alarm and warn others nearby as you are leaving. Your alertness can help save others.\n- Close doors and windows if time permits; this can help slow down the spread of fire and smoke.\n- Move away from the fire and smoke whenever possible. Use the stairs, not the elevators.\n- Feel the doors with the back of your hand** to check for heat; **do not open them if they are hot.\n- Do not re-enter the building** until instructed to do so by emergency responders. Trust their expertise to guide you safely.\n- Once outside, from a safe location, call the campus Police Department at 197\n\nif your evacuation route has excessive smoke and/or heat, or the exit door is w

In [52]:
#Pre-processing data

raw_doc= raw_doc.lower() #lower case conversion
nltk.download('punkt') #Tokenization
nltk.download('wordnet') #Downloads the WordNet database(only conatins english words)
nltk.download('omw-1.4') #Downloads the Open Multilingual WordNet (OMW)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [53]:
cleaned_doc = raw_doc.replace('\n\n', ' ').strip()

cleaned_doc

'fire in your workspace if you have a safe evacuation route (minimal or no fire and/or smoke) from the area: - stay calm and breathe deeply. remember that panic can make it harder to think clearly.\n- evacuate the building: to the designated emergency assembly area at a steady pace; there is no need to rush.\n- pull the fire alarm and warn others nearby as you are leaving. your alertness can help save others.\n- close doors and windows if time permits; this can help slow down the spread of fire and smoke.\n- move away from the fire and smoke whenever possible. use the stairs, not the elevators.\n- feel the doors with the back of your hand** to check for heat; **do not open them if they are hot.\n- do not re-enter the building** until instructed to do so by emergency responders. trust their expertise to guide you safely.\n- once outside, from a safe location, call the campus police department at 197 if your evacuation route has excessive smoke and/or heat, or the exit door is warm or ho

In [54]:
sentence_tokens = nltk.sent_tokenize(cleaned_doc)
word_tokens = nltk.word_tokenize(cleaned_doc)

print(sentence_tokens[:2])
print(word_tokens[0])

['fire in your workspace if you have a safe evacuation route (minimal or no fire and/or smoke) from the area: - stay calm and breathe deeply.', 'remember that panic can make it harder to think clearly.']
fire


In [55]:
#lemmatization
lemmer = nltk.stem.WordNetLemmatizer() #lemmatization using the WordNet database

def LemTokens(tokens):
  return[lemmer.lemmatize(token) for token in tokens]
#removing  punctuation
#Creates a translation dictionary that maps each punctuation character to None
#effectively removing it using translate()
remove_punc_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
  return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punc_dict)))

Define Greeting functions

In [56]:
greet_inputs = ('hello', 'hi', 'help', 'emergency', 'fire', 'assist', 'urgent')
greet_responses = (
    "Emergency services activated. How can I assist you?",
    "I'm here to help. What is the nature of your emergency?",
    "Hello, please provide details of your emergency.",
    "This is the Fire Emergency Bot. How can I assist you?",
    "Stay calm. I'm here to help. What's the situation?"
)

def greet(sentence):
    for word in sentence.split():
        if word.lower() in greet_inputs:
            return random.choice(greet_responses)
    return None

Response Generation

In [57]:
def response(user_input):
    bot_response = ''
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
 #tfidf is a matrix where each row represents a sentence and each column represents a term's TF-IDF score.
    tfidf = TfidfVec.fit_transform(sentence_tokens + [user_input])

    vals = cosine_similarity(tfidf[-1], tfidf[:-1])
    #This index corresponds to the most similar sentence to the user_input
    idx = vals.argsort()[0][-1]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-1]

    if req_tfidf == 0:
        bot_response = "I am sorry. Unable to understand you!"
    else:
        bot_response = sentence_tokens[idx]

    return bot_response

Defining the chatFlow

In [69]:
flag = True
print('Hello! I am FIREBOT. How can I assist you?')
while flag:
    user_input = input().lower()
    if user_input != 'bye':
        if user_input in ('thank you', 'thanks'):
            flag = True
            print('FIREBOT: You are welcome. Do you need more help?')
        else:
            if greet(user_input) is not None:
                print('FIREBOT: ' + greet(user_input))
            else:
                print('FIREBOT: ', end='')
                print(response(user_input))
    else:
        flag = False
        print('FIREBOT: Goodbye!')


Hello! I am FIREBOT. How can I assist you?
help
FIREBOT: Emergency services activated. How can I assist you?
can you give me the police number?
FIREBOT: - call the police department at 197 and let them know your exact location.
how can i evacuate?
FIREBOT: - evacuate the building: to the designated emergency assembly area at a steady pace; there is no need to rush.
what if i can't find a way out of the building?
FIREBOT: remember, help is on the way.
what if there's excessive smoke and heat  in my evacuation route?
FIREBOT: - once outside, from a safe location, call the campus police department at 197 if your evacuation route has excessive smoke and/or heat, or the exit door is warm or hot to the touch: - stay calm and stay put.
are elevators safe to use during a fire?
FIREBOT: about elevators - remember: elevators are not safe to use during a fire.
how can i focus on my breathing
FIREBOT: focus on your breathing: take slow, deep breaths to help calm your nerves.
what should i do to ke